Soo..

The idea here is to play a little bit with the "Frozen Lake" classic gym enviroment.

Here is a description from the open ai gym documentation 

https://github.com/openai/gym/blob/master/gym/envs/toy_text/frozen_lake.py

"""
    Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following
        
        SFFF
        FHFH
        FFFH
        HFFG
    
    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located
    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
"""

In [1]:
# Importing the libraries
import numpy as np
import gym
import time
import random

The actions available are:

LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

In [2]:
# Lets create our enviroment with Frozen Lake
env = gym.make("FrozenLake-v0", is_slippery=True)

As mentioned before the enviroment takes into account a "is_slippery" state, this creates a transition probability where you can take an action at a particular state but there is a probability you will end up in a different state of what your action intended you to go

In [3]:
# Lets define our initial states, states-values, discount factor, actions and threshold
V = np.zeros(env.nS)
S = np.arange(0,16)
threshold = 1e-3
gamma = 0.9
actions = {'LEFT':0,'DOWN':1,'RIGHT':2,'UP':3}

There are several ways to solve this problem. In this notebook i will use the policy iteration algorithm.

In [4]:
''' Lets first define a for policy evaluation'''
def policy_evaluation(env, V, S, policy):
    while True:
        delta = 0
        for s in S:
            v_old = V[s]
            expected_r = 0
            expected_v = 0
            transition = env.P[s][policy[s]]
            for (probs, state_prime, r, done) in transition:
                expected_r += probs * r
                expected_v += probs * V[state_prime]
            V[s] = expected_r + (gamma*expected_v)
            delta = max(delta, abs(v_old - V[s]))
        if delta <= threshold:
            break
    return(V)


In [5]:
''' Now lets create a function to generate a random policy and a function to generate policy improvements and iteration'''

def random_policy(env, S, actions):
    policy_random = {}
    for s in S:
        s_a = random.choice(list(actions.values()))
        policy_random[s] = s_a
    return(policy_random)


def policy_improvement_iteration(env, V, S):
    policy = random_policy(env, S, actions)
    while True:
        V = policy_evaluation(env, V, S, policy)
        new_policy = dict()
        for s in S:
            best_a = None
            best_p = float('-inf')
            for a in actions.values():
                expected_r = 0
                expected_v = 0
                transition = env.P[s][a]
                for (probs, state_prime, r, done) in transition:
                    expected_r += probs * r
                    expected_v += probs * V[state_prime]
                V_new = expected_r + (gamma*expected_v)
                if V_new >= best_p:
                    best_p = V_new
                    best_a = a
            new_policy[s] = best_a
        if new_policy == policy:
            break
        else:
            policy = new_policy
    return(policy)



Now we can run the algorithms and view the results

In [6]:
env.render()
start_time = time.time()
optimal_policy = policy_improvement_iteration(env, V, S)
print("--- %s seconds ---" % (time.time() - start_time))
optimal_actions = np.asarray(list(optimal_policy.values()))
print(optimal_actions.reshape(4,4), actions, sep='\n')



SFFF
FHFH
FFFH
HFFG
--- 0.004044771194458008 seconds ---
[[0 3 0 3]
 [0 3 2 3]
 [3 1 0 3]
 [3 2 1 3]]
{'LEFT': 0, 'DOWN': 1, 'RIGHT': 2, 'UP': 3}


As one can see, the actions at each state seem kind of counter-intuitive but thats because the "is_slippery" effect.

Let's calculate the results again but turning off the "is_slippery effect"

In [8]:
env2 = gym.make("FrozenLake-v0", is_slippery=False)
env2.render()
start_time = time.time()
optimal_policy = policy_improvement_iteration(env2, V, S)
print("--- %s seconds ---" % (time.time() - start_time))
optimal_actions = np.asarray(list(optimal_policy.values()))
print(optimal_actions.reshape(4,4), actions, sep='\n')


SFFF
FHFH
FFFH
HFFG
--- 0.0063838958740234375 seconds ---
[[2 2 1 0]
 [1 3 1 3]
 [2 2 1 3]
 [3 2 2 3]]
{'LEFT': 0, 'DOWN': 1, 'RIGHT': 2, 'UP': 3}


As one can see, without the "is_slippery" effect the calculated policy seems to be quite intuitive.